In [2]:
import pandas as pd
import json
import random
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')

# Парсинг json

### SQuAD

In [3]:
def load_json(file):
    data = json.load(f)
    data_par = json_normalize(data=data['data'], record_path=['paragraphs'], meta=['title'])
    
    return data_par

In [4]:
f = open('squad/train-v2.0.json')
squad_train = load_json(f)
f.close()

In [47]:
f = open('squad/dev-v2.0.json')
squad_dev = load_json(f)
f.close()

In [48]:
def json_to_df(data_par):
    new_df = pd.DataFrame()
    for idx, listOfDicts in data_par[['qas']].itertuples():
        qas = pd.DataFrame.from_dict(listOfDicts)
        qas['title'] = data_par['title'][idx]
        qas['context'] = data_par['context'][idx]
        qas['answer'] = ''
        qas['answer_start'] = ''
        for ind, answer in qas[['answers']].itertuples():
            if answer == []:
                answer = qas['plausible_answers'][ind]
            try:
                dic = random.sample(answer, 1)[0]
            except ValueError:
                pass
            qas['answer'][ind] = dic.get('text')
            qas['answer_start'][ind] = dic.get('answer_start')
        new_df = new_df.append(qas)

    new_df = new_df.drop('answers', axis=1)
    new_df = new_df.drop('plausible_answers', axis=1)   
    new_df = new_df.drop('id', axis=1)
    new_df = new_df.reset_index()
    new_df = new_df.drop('index', axis=1)
    
    return new_df

In [ ]:
squad_train = json_to_df(squad_train)
print(len(squad_train))

In [ ]:
squad_train.head(10)

In [49]:
squad_dev = json_to_df(squad_dev)
print(len(squad_dev))

11873


In [50]:
squad_dev.head(10)

,question,is_impossible,title,context,answer,answer_start
0,In what country is Normandy located?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,France,159
1,When were the Normans in Normandy?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,in the 10th and 11th centuries,87
2,From which countries did the Norse originate?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,"Denmark, Iceland and Norway",256
3,Who was the Norse leader?,False,Normans,The Normans (Norman: Nourmands; French: Norman...,Rollo,308
4,What century did the Normans first gain their ...,False,Normans,The Normans (Norman: Nourmands; French: Norman...,10th,671
5,Who gave their name to Normandy in the 1000's ...,True,Normans,The Normans (Norman: Nourmands; French: Norman...,Normans,4
6,What is France a region of?,True,Normans,The Normans (Norman: Nourmands; French: Norman...,Normandy,137
7,Who did King Charles III swear fealty to?,True,Normans,The Normans (Norman: Nourmands; French: Norman...,Rollo,308
8,When did the Frankish identity emerge?,True,Normans,The Normans (Norman: Nourmands; French: Norman...,10th century,671
9,Who was the duke in the battle of Hastings?,False,Normans,"The Norman dynasty had a major political, cult...",William the Conqueror,1022


### SberQuAD

In [18]:
f = open('sberquad/train-v1.1.json')
sberquad_train = load_json(f)
f.close()

In [44]:
f = open('sberquad/dev-v1.1.json')
sberquad_dev = load_json(f)
f.close()

In [43]:
def json_to_df_sber(data_par):
    new_df = pd.DataFrame({'context': data_par['context']})
    new_df['question'] = ''
    new_df['answer'] = ''
    new_df['answer_start'] = ''
    
    for idx, listOfDicts in data_par[['qas']].itertuples():
        dic = listOfDicts[0]
        new_df['question'][idx] = dic.get('question')
        answer = dic.get('answers')[0]
        new_df['answer'][idx] = answer.get('text')
        new_df['answer_start'][idx] = answer.get('answer_start')
    
    return new_df

In [ ]:
sberquad_train = json_to_df_sber(sberquad_train)
print(len(sberquad_train))

In [ ]:
sberquad_train.head(10)

In [45]:
sberquad_dev = json_to_df_sber(sberquad_dev)
print(len(sberquad_dev))

5036


In [46]:
sberquad_dev.head(10)

,context,question,answer,answer_start
0,"В Миссолонги Байрон заболел лихорадкой, продол...",Чем заболел Байрон в Миссолонги?,лихорадкой,28
1,От излишней потери тепла млекопитающих предохр...,Как отводятся излишки тепла у млекопитающих?,потоотделением,132
2,Хромосомные перестройки (хромосомные аберрации...,Что нарушают хромосомные аберрации?,Структуру хромосом.,74
3,Первый в мире городской автобус с двигателем в...,В каком городе первый в мире городской автобус...,в Лондоне,103
4,Фродом являются также и операции с поддельными...,В каком режиме проходят подлимитные операции?,в режиме офлайн,455
5,Слоговое письмо представляет собой набор письм...,Находит ли отражение в алфавите фонетическая с...,не находит,368
6,Весной 1890 года художник перебрался в Овер-сю...,Каким стал стиль последних работ художника?,нервным и гнетущим,232
7,В 2008 году родилось 1147 человек у граждан Шв...,Сколько человек родилось в 2008 году у граждан...,1147 человек,21
8,Иногда говорят о возможности провести GNU GPL ...,Какая статья и пункт дает возможность статья д...,пункт 2 статьи 434,558
9,Первым пособием по товароведению в России обыч...,Какими русскими купцами был накоплен значимый ...,купцами-новгородцами,245
